# **(1)** Notebook Setup
* Import packages
* Set file path, user name, email
* Connect to WRDS — You will be asked for your log in information

In [82]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import requests
from io import StringIO

import time
from tqdm import tqdm

from fuzzywuzzy import fuzz

# NOTE: update the below with the path to your preferred directory
path = "/Users/dorothydickmann/Desktop/GitHubRepos/DeweyExplore/DataFiles"

# NOTE: update the below with your own name and email address
# this is required to pull data from SEC EDGAR.
dl_user_name = 'Dorothy Dickmann'
dl_user_email = 'dorothy.dickmann@gmail.com'

In [83]:
# set up WRDS connection
import wrds
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [84]:
# Function to find best fuzzy match
# Setting minimum score to 80; up to user discretion.
def find_best_match(brand_name, company_list, min_score=85):
    best_score = 0
    best_match = None
    
    for _, company in company_list.iterrows():
        score = fuzz.token_sort_ratio(str(brand_name), str(company['Company Name']))
        if score > best_score:
            best_score = score
            best_match = company
    
    if best_score >= min_score:
        return pd.Series({
            'sec_company_name': best_match['Company Name'],
            'sec_cik': best_match['CIK'],
            'fuzzy_score_sec': best_score
        })
    return pd.Series({
        'sec_company_name': None,
        'sec_cik': None,
        'fuzzy_score_sec': None,
    })

#  **(2)** Get data for Compustat Linking
* Pull compustat unique ID data for fiscal years 2019 and beyond 
    * *Note: SG data starts in 2019.*
* Not including `cusip`, because the best way to link to CRSP is using the CRSP-Compustat link table to get the CRSP `permno`

## *Output*
*`uniqueTickers` &rarr; dataframe of unique compustat companies.*

*Columns:*
* *`gvkey, tic, conm, cik`*

In [85]:
# note: we're not including CUSIP here because
# Compustat only tracks the cusip for the "primary security" of the company

compustat = conn.raw_sql("""select datadate, gvkey, tic, cik, 
                         conm, exchg, sich, fyear
                    from comp.funda
                    where datadate >= '2019-01-01'
                    and consol = 'C'
                    and indfmt = 'INDL'
                    and datafmt = 'STD'
                    and popsrc = 'D'
                    and curcd = 'USD'
                    and costat = 'A'
                    """)
compustat.head(2)

,datadate,gvkey,tic,cik,conm,exchg,sich,fyear
0,2019-05-31,001004,AIR,0000001750,AAR CORP,11,5080.0,2018
1,2020-05-31,001004,AIR,0000001750,AAR CORP,11,5080.0,2019


In [86]:
uniqueTickers = compustat[['gvkey','tic','conm','cik']].drop_duplicates().copy()
print(uniqueTickers.shape[0])
uniqueTickers.head(2)

11096


,gvkey,tic,conm,cik
0,001004,AIR,AAR CORP,0000001750
6,001045,AAL,AMERICAN AIRLINES GROUP INC,0000006201


In [87]:
# strip leading zeros from cik, handle None values, and turn into a number
uniqueTickers['cik'] = uniqueTickers['cik'].astype(str).str.lstrip('0')
uniqueTickers.loc[uniqueTickers['cik'] == 'None', 'cik'] = np.nan
uniqueTickers['cik'] = uniqueTickers['cik'].astype(float)

In [88]:
# there's one duplicate company name, but since we're using tic, we want to keep it.
# there are fewer ciks than gvkeys/tickers because many compustat companies don't have ciks (e.g. international)
print(f"Number of unique tickers: {uniqueTickers.drop_duplicates(subset=['tic']).shape[0]}")
print(f"Number of unique company names: {uniqueTickers.drop_duplicates(subset=['conm']).shape[0]}")
print(f"Number of unique gvkeys: {uniqueTickers.drop_duplicates(subset=['gvkey']).shape[0]}")
print(f"Number of unique ciks: {uniqueTickers.drop_duplicates(subset=['cik']).shape[0]}")

Number of unique tickers: 11096
Number of unique company names: 11095
Number of unique gvkeys: 11096
Number of unique ciks: 7089


# **(3)** Prep SafeGraph Brand Info
* Pull in data from the [SafeGraph Brand](https://app.deweydata.io/products/5a88b56e-1155-4784-bba6-c0f4afd9b6ef/package) table
* Partition into two dataframes — One with brands that have a `STOCK_SYMBOL` (ticker), and one without.

## *Output*
*`brandInfo` &rarr; dataframe of all SafeGraph Brand data*

*`sgTickerData` &rarr; dataframe of unique brands and their stock symbols. does not contain brands with null stock symbols.*

* *Columns:*
    * *`SAFEGRAPH_BRAND_ID, PARENT_SAFEGRAPH_BRAND_ID BRAND_NAME, STOCK_SYMBOL, STOCK_EXCHANGE`*

*`sgNoTickerData` &rarr; dataframe of unique brands without stock symbols*

* *Columns:*
    * *`SAFEGRAPH_BRAND_ID, PARENT_SAFEGRAPH_BRAND_ID, BRAND_NAME, PARENT_SAFEGRAPH_BRAND_ID`*

In [89]:
# read in SafeGraphBrandInfo.csv
brandInfo = pd.read_csv(f'{path}/SafeGraphBrandInfo.csv')
brandInfo.head()


,SAFEGRAPH_BRAND_ID,BRAND_NAME,PARENT_SAFEGRAPH_BRAND_ID,NAICS_CODE,TOP_CATEGORY,SUB_CATEGORY,STOCK_SYMBOL,STOCK_EXCHANGE,ISO_COUNTRY_CODES_OPEN,ISO_COUNTRY_CODES_CLOSED
0,SG_BRAND_ca78258dd349e2cd,Cargill Manufacturing,SG_BRAND_53546b685daf0f35,3119,Other Food Manufacturing,NaN,NaN,NaN,"""[\""US\""]""","""[]"""
1,SG_BRAND_53546b685daf0f35,Cargill,NaN,3119,Other Food Manufacturing,NaN,NaN,NaN,"""[]""","""[]"""
2,SG_BRAND_7be85c7fa2bf01b1,Avery Dennison Distribution,SG_BRAND_15caf5d4c87a50eb,3222,Converted Paper Product Manufacturing,NaN,NaN,NaN,"""[\""US\""]""","""[]"""
3,SG_BRAND_49790effc276c9cc,Avery Dennison Corporate,SG_BRAND_15caf5d4c87a50eb,3222,Converted Paper Product Manufacturing,NaN,NaN,NaN,"""[\""US\""]""","""[]"""
4,SG_BRAND_15caf5d4c87a50eb,Avery Dennison Corporation,NaN,3222,Converted Paper Product Manufacturing,NaN,NaN,NaN,"""[]""","""[]"""


In [90]:
print(f"Total rows: {brandInfo.shape[0]}")
print(f"Unique brand ids: {brandInfo.drop_duplicates('SAFEGRAPH_BRAND_ID').shape[0]}")
print(f"Unique brand names: {brandInfo.drop_duplicates('BRAND_NAME').shape[0]}")
print(f"Rows with stock symbol data: {brandInfo['STOCK_SYMBOL'].notna().sum()}")
print(f"Unique stock symbols: {brandInfo.drop_duplicates('STOCK_SYMBOL').shape[0]}")

Total rows: 15199
Unique brand ids: 15199
Unique brand names: 15199
Rows with stock symbol data: 3038
Unique stock symbols: 1111


In [91]:
# sanity check:
# let's look at one ticker that should have multiple brands
# ANF = Abercrombie & Fitch (owns both Hollister and Abercrombie brands)
# so we should have 3 rows for ANF
brandInfo.loc[brandInfo['STOCK_SYMBOL'] == 'ANF']

,SAFEGRAPH_BRAND_ID,BRAND_NAME,PARENT_SAFEGRAPH_BRAND_ID,NAICS_CODE,TOP_CATEGORY,SUB_CATEGORY,STOCK_SYMBOL,STOCK_EXCHANGE,ISO_COUNTRY_CODES_OPEN,ISO_COUNTRY_CODES_CLOSED
13507,SG_BRAND_4f6c6d3220f6da7d783d2d378ea2d4c4,Hollister,SG_BRAND_6d8684d641d51f2d,448140,Clothing Stores,Family Clothing Stores,ANF,NYSE,"""[\""MX\"",\""ID\"",\""JP\"",\""CN\"",\""IE\"",\""QA\"",\""...","""[\""MX\"",\""CN\"",\""ES\"",\""DE\"",\""GB\"",\""IT\"",\""..."
13531,SG_BRAND_6d8684d641d51f2d,Abercrombie & Fitch Co.,NaN,448140,Clothing Stores,Family Clothing Stores,ANF,NYSE,"""[]""","""[]"""
13539,SG_BRAND_7cced0ecbfbf09fc,Abercrombie,SG_BRAND_6d8684d641d51f2d,448140,Clothing Stores,Family Clothing Stores,ANF,NYSE,"""[\""MX\"",\""ID\"",\""JP\"",\""CN\"",\""QA\"",\""ES\"",\""...","""[\""CN\"",\""ES\"",\""DE\"",\""GB\"",\""IT\"",\""FR\"",\""..."


In [92]:
# get SG data that has a non-null `STOCK_SYMBOL`
sgTickerData = brandInfo.loc[brandInfo['STOCK_SYMBOL'].notna()]
# and the data wihtout stock symbols.
sgNoTickerData = brandInfo.loc[brandInfo['STOCK_SYMBOL'].isna()]

# let's limit it to just the SAFEGRAPH_BRAND_ID, BRAND_NAME, STOCK_SYMBOL, and STOCK_EXCHANGE
sgTickerData = sgTickerData[['SAFEGRAPH_BRAND_ID', 'BRAND_NAME', 'STOCK_SYMBOL', 'STOCK_EXCHANGE', 'PARENT_SAFEGRAPH_BRAND_ID']].copy()
sgNoTickerData = sgNoTickerData[['SAFEGRAPH_BRAND_ID', 'BRAND_NAME', 'PARENT_SAFEGRAPH_BRAND_ID']].copy()

# get the number of unique `STOCK_SYMBOL`
print(f"Unique stock symbols: {sgTickerData['STOCK_SYMBOL'].drop_duplicates().shape[0]}")

Unique stock symbols: 1110


In [93]:
sgTickerData.sample(5) 

,SAFEGRAPH_BRAND_ID,BRAND_NAME,STOCK_SYMBOL,STOCK_EXCHANGE,PARENT_SAFEGRAPH_BRAND_ID
12412,SG_BRAND_2acbfc956fcbdd22,Ashworth Golf,0375,HKG,NaN
12438,SG_BRAND_279d7e214ee90460,Tao Heung Tea House,0573,HKG,SG_BRAND_6fdf851409ec419c
13202,SG_BRAND_30c522a303739f62,Vista Outdoor,VSTO,NYSE,NaN
14842,SG_BRAND_99fbcd9a49a4f44e,Naturalia,CGUSY,OTCMKTS,SG_BRAND_351f7fb0ba8e4ee1
13075,SG_BRAND_3a68cf72e51191c3,Kellogg Manufacturing,K,NYSE,SG_BRAND_efb04872cd564db5


# **(4)** Merge Together on Ticker
* `left` merge the SafeGraph Brand data with Compustat data on `STOCK_SYMBOL` / `tic`
* Identify match type as `ticker`

## *Output*
*`sg_compustat` &rarr; `sgTickerData` merged with `uniqueTickers` on `STOCK_SYMBOL` and `tic`, left join.

* *Columns:*
    * *`SAFEGRAPH_BRAND_ID, SAFEGRAPH_PARENT_BRAND_ID, BRAND_NAME, STOCK_SYMBOL, STOCK_EXCHANGE, gvkey, tic, conm, cik`*

* *Approx. 60% match rate overall, with > 90% match rate for NYSE and NASDAQ companies*

In [94]:
# merge sgTickerData with uniqueTickers on STOCK_SYMBOL and tic, left join.

sg_compustat = sgTickerData.merge(uniqueTickers, 
                            left_on='STOCK_SYMBOL', 
                            right_on='tic', 
                            how='left')

In [95]:
# let's look at the coverage by stock exchange
# NOTE: as one would expect, there's pretty good coverage for NYSE and NASDAQ. 
sg_compustat.groupby('STOCK_EXCHANGE').count().sort_values('gvkey', ascending=False).head(10)

,SAFEGRAPH_BRAND_ID,BRAND_NAME,STOCK_SYMBOL,PARENT_SAFEGRAPH_BRAND_ID,gvkey,tic,conm,cik
STOCK_EXCHANGE,,,,,,,,
NYSE,1110,1110,1110,691,1043,1043,1043,1043
NASDAQ,526,526,526,273,477,477,477,474
OTCMKTS,469,469,469,327,102,102,102,98
TSX,196,196,196,154,44,44,44,37
LSE,156,156,156,98,39,39,39,37
EPA,41,41,41,26,20,20,20,20
ASX,76,76,76,45,18,18,18,10
BMAD,12,12,12,9,12,12,12,7
SWX,20,20,20,9,11,11,11,11


In [96]:
print(f"SG brands without gvkey: {sg_compustat['gvkey'].isna().sum()}")
print(f"SG brands with gvkey: {sg_compustat['gvkey'].notna().sum()}")
print(f"pct of SG brands with gvkey: {sg_compustat['gvkey'].notna().mean():.2%}")

SG brands without gvkey: 1232
SG brands with gvkey: 1806
pct of SG brands with gvkey: 59.45%


In [97]:
print(f"SG brands without cik: {sg_compustat['cik'].isna().sum()}")
print(f"SG brands with cik: {sg_compustat['cik'].notna().sum()}")
print(f"pct of SG brands with cik: {sg_compustat['cik'].notna().mean():.2%}")

SG brands without cik: 1270
SG brands with cik: 1768
pct of SG brands with cik: 58.20%


In [98]:
# create a match type column to describe the type of match we make.
sg_compustat.loc[sg_compustat['gvkey'].notnull(),'match_type'] = 'ticker'
sg_compustat.groupby('match_type').count()

,SAFEGRAPH_BRAND_ID,BRAND_NAME,STOCK_SYMBOL,STOCK_EXCHANGE,PARENT_SAFEGRAPH_BRAND_ID,gvkey,tic,conm,cik
match_type,,,,,,,,,
ticker,1806,1806,1806,1806,1089,1806,1806,1806,1768


# **(5)** Fuzzy Match with SEC data

* Get detailed listing of all SEC filings ("master index") for 2019 - 2024
* Limit to companies that have reported a 10-K (are reporting companies)
* Drop duplicate companies, keep only company name and cik
* Execute fuzzy match between `BRAND_NAME` and `Company Name`

### *Output*
`master_index`, `master_index_2019_2024.csv` &rarr; dataframe of SEC master index for 2019 - 2024


`sg_compustat` &rarr; dataframe of SafeGraph Brands with Compustat matches

* *Columns:*
    * *`SAFEGRAPH_BRAND_ID, SAFEGRAPH_PARENT_BRAND_ID, BRAND_NAME, STOCK_SYMBOL, STOCK_EXCHANGE, gvkey, conm, match_type, sec_company_name, fuzzy_score_sec, matched_cik`*

In [99]:
# can comment this section out if you don't want to pull a fresh master index
master_index = pd.DataFrame()

for year in range(2019,2025):
    print('working on ', year)
    for quarter in range(1,5):
        # Create a variable that contains the URL of the index we want to access
        index_url = f"https://www.sec.gov/Archives/edgar/full-index/{year}/QTR{quarter}/company.idx"

        # Download the content from the URL above
        index_content = requests.get(index_url, headers={'User-Agent': f'{dl_user_name} {dl_user_email}'}).content

        # Create a DataFrame from the content downloaded in the previous cell
        index_df = pd.read_fwf(StringIO(index_content.decode('ISO-8859-1')), skiprows=(0,1,2,3,4,5,6,7,9), widths = [62,12,12,12,52])

        # Add a year variable
        index_df['year'] = f"{year}"

        # Append the dataframes
        master_index = pd.concat([master_index,index_df])
        index_df.to_parquet(f'{path}/Raw_Index_Y{year}Q{quarter}.parquet', index = False)
        print('saved: ', year, quarter)

master_index['url'] = 'https://www.sec.gov/Archives/' + master_index['File Name']
master_index.to_parquet(f'{path}/master_index_2019_2024.csv', index = False)

working on  2019
saved:  2019 1
saved:  2019 2
saved:  2019 3
saved:  2019 4
working on  2020
saved:  2020 1
saved:  2020 2
saved:  2020 3
saved:  2020 4
working on  2021
saved:  2021 1
saved:  2021 2
saved:  2021 3
saved:  2021 4
working on  2022
saved:  2022 1
saved:  2022 2
saved:  2022 3
saved:  2022 4
working on  2023
saved:  2023 1
saved:  2023 2
saved:  2023 3
saved:  2023 4
working on  2024
saved:  2024 1
saved:  2024 2
saved:  2024 3
saved:  2024 4


In [100]:
# limit to reporting companies
reportingCompanies = master_index.loc[master_index['Form Type'] == '10-K']
reportingCompanies = reportingCompanies[['CIK','Company Name']].drop_duplicates()
reportingCompanies.head()


,CIK,Company Name
113,1591890,"1347 Property Insurance Holdings, Inc."
300,1141807,1ST CONSTITUTION BANCORP
315,34782,1ST SOURCE CORP
354,38723,1st FRANKLIN FINANCIAL CORP
457,1347858,"22nd Century Group, Inc."


In [101]:
# turn CIK into a number (float)
reportingCompanies['CIK'] = reportingCompanies['CIK'].astype(float)

In [102]:
# check amount of time it will take
# Test execution time on a sample

# Create small samples for testing
sample_sg = sg_compustat.head(10)  # Take 10 records

# Time the sample matching
start_time = time.time()

# Run the sample matching
for _, row in sample_sg.iterrows():
    _ = find_best_match(row['BRAND_NAME'], reportingCompanies)

end_time = time.time()

# Calculate metrics
time_per_comparison = (end_time - start_time) / (len(sample_sg) * len(reportingCompanies))
total_comparisons_needed = len(sg_compustat) * len(reportingCompanies)
estimated_total_time = time_per_comparison * total_comparisons_needed

# Print results
print(f"Sample execution time: {end_time - start_time:.2f} seconds")
print(f"Time per comparison: {time_per_comparison:.6f} seconds")
print(f"\nFull matching would require:")
print(f"Number of SafeGraph records: {len(sg_compustat):,}")
print(f"Number of SEC companies: {len(reportingCompanies):,}")
print(f"Total comparisons needed: {total_comparisons_needed:,}")
print(f"\nEstimated total time:")
print(f"Minutes: {estimated_total_time/60:.1f}")
print(f"Hours: {estimated_total_time/3600:.1f}")

Sample execution time: 2.26 seconds
Time per comparison: 0.000019 seconds

Full matching would require:
Number of SafeGraph records: 3,038
Number of SEC companies: 11,625
Total comparisons needed: 35,316,750

Estimated total time:
Minutes: 11.4
Hours: 0.2


In [103]:
# Apply fuzzy matching to all companies
fuzzy_matches = []
for _, row in tqdm(sg_compustat.iterrows(), total=len(sg_compustat), desc="Fuzzy matching"):
    match = find_best_match(row['BRAND_NAME'], reportingCompanies)
    fuzzy_matches.append(match)

# Convert list of matches to DataFrame and add to original DataFrame
fuzzy_matches = pd.DataFrame(fuzzy_matches)
sg_compustat = pd.concat([sg_compustat, fuzzy_matches], axis=1)

# Show results
print("\nMatching Results:")
print(f"Total companies: {len(sg_compustat)}")
print(f"Fuzzy matches found: {sg_compustat['fuzzy_score_sec'].notna().sum()}")

Fuzzy matching: 100%|██████████| 3038/3038 [10:51<00:00,  4.66it/s]


Matching Results:
Total companies: 3038
Fuzzy matches found: 217


In [104]:
# create a column called "matched_cik"
# first, set it equal to "cik"
# then, if "gvkey" is null (there is no compustat data), set it equal to "sec_cik"
sg_compustat['matched_cik'] = sg_compustat['cik']
sg_compustat.loc[sg_compustat['gvkey'].isnull(), 'matched_cik'] = sg_compustat['sec_cik']

# if cik is null and sec_cik is not null, set match_type to "company name"
sg_compustat.loc[(sg_compustat['gvkey'].isnull()) & 
                 (sg_compustat['sec_cik'].notnull()), 'match_type'] = 'company name'

In [105]:
# if match_type is "ticker"
# set the folllowing columns to null values: fuzzy_score_sec, sec_company_name
# I want to preserve the fuzzy score and the company name for those where we're 
# using the company name match, but not for those where we're using the ticker match.
sg_compustat.loc[sg_compustat['match_type'] == 'ticker', 'fuzzy_score_sec'] = None
sg_compustat.loc[sg_compustat['match_type'] == 'ticker', 'sec_company_name'] = None


In [106]:
# drop extra columns
# tic, cik, sec_cik
sg_compustat.drop(columns=['tic', 'cik', 'sec_cik'], inplace=True)


In [107]:
sg_compustat.sample(10)

,SAFEGRAPH_BRAND_ID,BRAND_NAME,STOCK_SYMBOL,STOCK_EXCHANGE,PARENT_SAFEGRAPH_BRAND_ID,gvkey,conm,match_type,sec_company_name,fuzzy_score_sec,matched_cik
2756,SG_BRAND_ef23ef1d426ae1a73a7eed399cef707d,COS,HNNMY,OTCMKTS,SG_BRAND_80f13a64baa85d53,NaN,NaN,NaN,None,NaN,NaN
1596,SG_BRAND_09eda2f408aefcc9,BankUnited ATM,BKU,NYSE,SG_BRAND_850a8eaee850fbc06e74a5e3708d247b,185824,BANKUNITED INC,ticker,None,NaN,1504008.0
381,SG_BRAND_beafc4df9142d172,M&S Hearing,MKS,LSE,SG_BRAND_6038b20dccfd4a8c,NaN,NaN,NaN,None,NaN,NaN
134,SG_BRAND_b61587f11ab706cd,EDEN Laser Clinics,SLA,ASX,SG_BRAND_35bd527c3ac37f08,NaN,NaN,NaN,None,NaN,NaN
2108,SG_BRAND_6e7bcf9086fc3b43babdfdf51a97759f,Bed Bath & Beyond,BBBY,NASDAQ,NaN,NaN,NaN,company name,BED BATH & BEYOND INC,88.0,886158.0
1069,SG_BRAND_25225d143d137db1920f737ceb6c9ee2,Johnson Controls,JCI,NYSE,NaN,006268,JOHNSON CONTROLS INTL PLC,ticker,None,NaN,833444.0
2689,SG_BRAND_e36d85e97a3924170277fdc8e0df0453,PriceRite,SRHGF,OTCMKTS,SG_BRAND_ce8c4f0fd8517b73,NaN,NaN,NaN,None,NaN,NaN
1926,SG_BRAND_51ea13820ab33db608fdc6af3f8186e4,MGM Resorts,MGM,NYSE,NaN,014418,MGM RESORTS INTERNATIONAL,ticker,None,NaN,789570.0
2996,SG_BRAND_77b8fc8a14e6a7dc,Scentre Group,STGPF,OTCMKTS,NaN,NaN,NaN,NaN,None,NaN,NaN
1296,SG_BRAND_8d819ffb9fa80295260ecf779927ef25,Sunoco,SUN,NYSE,NaN,012892,SUNOCO LP,ticker,None,NaN,1552275.0


In [108]:
sg_compustat.to_csv(f'{path}/SafeGraph_PublicCompanies_WithSGTicker.csv')

In [109]:
# Take a look at the percentage match rate for NYSE, NASDAQ, and OTCMKTS
# NOTE: OTCMKTS is not actually a stock exchange, it's a market for over-the-counter stocks
major_exchanges = ['NYSE', 'NASDAQ', 'OTCMKTS']
summary = sg_compustat.loc[sg_compustat['STOCK_EXCHANGE'].isin(major_exchanges),
                 ['STOCK_EXCHANGE','SAFEGRAPH_BRAND_ID','gvkey','matched_cik']].groupby('STOCK_EXCHANGE').count()
summary.rename(columns = {'SAFEGRAPH_BRAND_ID':'total_brands'}, inplace = True)
summary['pct_cik_match'] = summary['matched_cik'] / summary['total_brands']
summary['pct_gvkey_match'] = summary['gvkey'] / summary['total_brands']

summary

,total_brands,gvkey,matched_cik,pct_cik_match,pct_gvkey_match
STOCK_EXCHANGE,,,,,
NASDAQ,526,477,481,0.914449,0.906844
NYSE,1110,1043,1049,0.945045,0.939640
OTCMKTS,469,102,99,0.211087,0.217484


# **(6)** Fuzzy Match SafeGraph Brands without a `STOCK_SYMBOL` to SEC Company Name

## *Output*
`sgNoTickerData` &rarr; dataframe of SafeGraph Brands without a `STOCK_SYMBOL` with SEC Company Name matches

* *Columns:*
    * *`SAFEGRAPH_BRAND_ID, match_type, gvkey, conm, matched_cik, sec_company_name, fuzzy_score_sec`*


In [110]:
# Create small samples for testing
sample_sg = sgNoTickerData.head(100)  # Take 100 records
sample_companies = reportingCompanies  # Use full reporting companies list

# Time the sample matching
start_time = time.time()

# Run the sample matching
for _, row in sample_sg.iterrows():
    _ = find_best_match(row['BRAND_NAME'], sample_companies)

end_time = time.time()

# Calculate metrics
time_per_comparison = (end_time - start_time) / (len(sample_sg) * len(sample_companies))
total_comparisons_needed = len(sgNoTickerData) * len(reportingCompanies)
estimated_total_time = time_per_comparison * total_comparisons_needed

# Print results
print(f"Sample execution time: {end_time - start_time:.2f} seconds")
print(f"Time per comparison: {time_per_comparison:.6f} seconds")
print(f"\nFull matching would require:")
print(f"Number of SafeGraph records: {len(sgNoTickerData):,}")
print(f"Number of SEC companies: {len(reportingCompanies):,}")
print(f"Total comparisons needed: {total_comparisons_needed:,}")
print(f"\nEstimated total time:")
print(f"Minutes: {estimated_total_time/60:.1f}")

Sample execution time: 21.04 seconds
Time per comparison: 0.000018 seconds

Full matching would require:
Number of SafeGraph records: 12,161
Number of SEC companies: 11,625
Total comparisons needed: 141,371,625

Estimated total time:
Minutes: 42.6


In [111]:
# Apply fuzzy matching to all companies without stock info
print("Starting fuzzy matching process...")
fuzzy_matches = []
for _, row in tqdm(sgNoTickerData.iterrows(), total=len(sgNoTickerData), desc="Fuzzy matching"):
    match = find_best_match(row['BRAND_NAME'], reportingCompanies)
    fuzzy_matches.append(match)

# Convert list of matches to DataFrame and add to original DataFrame
fuzzy_matches = pd.DataFrame(fuzzy_matches)
sgNoTickerData = pd.concat([sgNoTickerData, fuzzy_matches], axis=1)

# Show results
print("\nMatching Results:")
print(f"Total companies: {len(sgNoTickerData)}")
print(f"Fuzzy matches found: {sgNoTickerData['fuzzy_score_sec'].notna().sum()}")

Starting fuzzy matching process...


Fuzzy matching: 100%|██████████| 12161/12161 [42:50<00:00,  4.73it/s]



Matching Results:
Total companies: 12163
Fuzzy matches found: 44


In [112]:
# rename the sec_cik column to matched_cik
sgNoTickerData.rename(columns={'sec_cik':'matched_cik'}, inplace=True)

# create a column called "match_type"
# if fuzzy_score_sec is not null, set it equal to "company name"
# if null, set set match_type to null
sgNoTickerData['match_type'] = np.where(sgNoTickerData['fuzzy_score_sec'].notnull(), 'company name', None)

In [113]:
# save the results
sgNoTickerData.to_csv(f'{path}/SafeGraph_PublicCompanies_WithoutSGTicker.csv', index = False)

# **(7)** `PARENT_SAFEGRAPH_BRAND_ID` matching

* Some SafeGraph Brands could have a `PARENT_SAFEGRAPH_BRAND_ID` that is a public company, but may not be identified with a `STOCK_SYMBOL`, or their parent company may have matched via SEC Company Name.

* Here we'll:
    1. Concatenate the two datasets
    2. Create a dataframe of SafeGraph Brands with public company matches
    3. Merge the two datasets on `PARENT_SAFEGRAPH_BRAND_ID` and `SAFEGRAPH_BRAND_ID`

## *Output*
* `allBrands` &rarr; dataframe of all SafeGraph Brands with public company matches, including parent company matches

* *Columns:*
    * *`SAFEGRAPH_BRAND_ID, match_type, gvkey, conm, matched_cik, sec_company_name, fuzzy_score_sec`*


In [114]:
brandInfo.columns.values

array(['SAFEGRAPH_BRAND_ID', 'BRAND_NAME', 'PARENT_SAFEGRAPH_BRAND_ID',
       'NAICS_CODE', 'TOP_CATEGORY', 'SUB_CATEGORY', 'STOCK_SYMBOL',
       'STOCK_EXCHANGE', 'ISO_COUNTRY_CODES_OPEN',
       'ISO_COUNTRY_CODES_CLOSED'], dtype=object)

In [152]:
allBrands = pd.concat([sg_compustat, sgNoTickerData])

allBrands.columns.values

array(['SAFEGRAPH_BRAND_ID', 'BRAND_NAME', 'STOCK_SYMBOL',
       'STOCK_EXCHANGE', 'PARENT_SAFEGRAPH_BRAND_ID', 'gvkey', 'conm',
       'match_type', 'sec_company_name', 'fuzzy_score_sec', 'matched_cik'],
      dtype=object)

In [153]:
# create a dataframe of SafeGraph Brands with public company matches
matchedCos = allBrands.loc[allBrands['match_type'].notnull()].copy()

matchedCos = matchedCos[['SAFEGRAPH_BRAND_ID', 'match_type','gvkey', 'conm',
                         'matched_cik','sec_company_name','fuzzy_score_sec']].copy()

# Rename columns to add _par suffix, except for SAFEGRAPH_BRAND_ID
matchedCos.rename(columns={
    'match_type': 'match_type_par',
    'gvkey': 'gvkey_par', 
    'conm': 'conm_par',
    'matched_cik': 'matched_cik_par',
    'sec_company_name': 'sec_company_name_par',
    'fuzzy_score_sec': 'fuzzy_score_sec_par'
}, inplace = True)

print(allBrands.shape[0])
allBrands = pd.merge(allBrands, matchedCos,
                     left_on = 'PARENT_SAFEGRAPH_BRAND_ID',
                     right_on = 'SAFEGRAPH_BRAND_ID',
                     how = 'left')
print(allBrands.shape[0])

15201
15201


In [154]:
allBrands.columns.values

array(['SAFEGRAPH_BRAND_ID_x', 'BRAND_NAME', 'STOCK_SYMBOL',
       'STOCK_EXCHANGE', 'PARENT_SAFEGRAPH_BRAND_ID', 'gvkey', 'conm',
       'match_type', 'sec_company_name', 'fuzzy_score_sec', 'matched_cik',
       'SAFEGRAPH_BRAND_ID_y', 'match_type_par', 'gvkey_par', 'conm_par',
       'matched_cik_par', 'sec_company_name_par', 'fuzzy_score_sec_par'],
      dtype=object)

In [155]:
# drop SAFEGRAPH_BRAND_ID_y, and rename SAFEGRAPH_BRAND_ID_x to SAFEGRAPH_BRAND_ID
allBrands.drop(columns=['SAFEGRAPH_BRAND_ID_y'], inplace = True)
allBrands.rename(columns={'SAFEGRAPH_BRAND_ID_x':'SAFEGRAPH_BRAND_ID'}, inplace = True)

In [156]:
allBrands.loc[(allBrands['match_type'].isnull()) & 
              (allBrands['match_type_par'].notnull())].shape

(137, 17)

In [157]:
# for those that don't have an origial match, but have a parent match,
# set match_type to the value of match_type_par
allBrands.loc[(allBrands['match_type'].isnull()) & 
              (allBrands['match_type_par'].notnull()),'match_type'] = allBrands['match_type_par']

# do the same for matched_cik, gvkey, sec_company_name, company name, and fuzzy_score_sec
allBrands.loc[(allBrands['match_type'].isnull()) & 
              (allBrands['match_type_par'].notnull()),'matched_cik'] = allBrands['matched_cik_par']


allBrands.loc[(allBrands['match_type'].isnull()) & 
              (allBrands['match_type_par'].notnull()),'gvkey'] = allBrands['gvkey_par']


allBrands.loc[(allBrands['match_type'].isnull()) & 
              (allBrands['match_type_par'].notnull()),'conm'] = allBrands['conm_par']

allBrands.loc[(allBrands['match_type'].isnull()) & 
              (allBrands['match_type_par'].notnull()),'sec_company_name'] = allBrands['sec_company_name_par']

allBrands.loc[(allBrands['match_type'].isnull()) & 
              (allBrands['match_type_par'].notnull()),'fuzzy_score_sec'] = allBrands['fuzzy_score_sec_par']

# drop the match_type_par, matched_cik_par, gvkey_par, sec_company_name_par, and fuzzy_score_sec_par columns
allBrands.drop(columns=['match_type_par', 'matched_cik_par', 
                        'gvkey_par', 'sec_company_name_par', 
                        'fuzzy_score_sec_par', 'conm_par'], inplace = True)


In [158]:
allBrands.to_csv(f'{path}/SafeGraph_PublicCompanies.csv', index = False)

In [159]:
allBrands.columns.values

array(['SAFEGRAPH_BRAND_ID', 'BRAND_NAME', 'STOCK_SYMBOL',
       'STOCK_EXCHANGE', 'PARENT_SAFEGRAPH_BRAND_ID', 'gvkey', 'conm',
       'match_type', 'sec_company_name', 'fuzzy_score_sec', 'matched_cik'],
      dtype=object)